In [1]:
import sys, os

ii = sys.argv[1]
info_file = file("CGI_classification_revised.txt")
info_line = info_file.readline().strip()
info_list = []
for i in range(1,25):
    info_list.append([])
    
while info_line:
    info_split = info_line.split('\t')
    info_chr = info_split[1]
    info_index = int(info_chr.replace("chr","").replace("X","23").replace("Y","24")) - 1
    info_start = int(info_split[3]) + 1
    info_end = int(info_split[4])
    info_CGI = info_chr + ':' + info_split[3]
    
    info_list[info_index].append([info_CGI,info_start,info_end,info_split[6],info_split[12]])
    info_line = info_file.readline().strip()

print 'info_list has been completed'
info_file.close()

for i in range(int(ii),int(ii)+1):
    print 'chr' + str(i).replace("23","X").replace("24","Y")
    input_file = file("revision_CpG_CtoT_PCAWG_chr%s.txt" %str(i).replace("23","X").replace("24","Y"))
    output_file = file("revision_CpG_CtoT_PCAWG_CGI_reclassification_chr%s.txt" %str(i).replace("23","X").replace("24","Y"),"w")
    
    input_line = input_file.readline().strip()
    output_file.write(input_line+'\tIn_CGI\tnear_CGI\tdistace_from_CGI\tCGI_size\tCGI_class\n')
    
    input_line = input_file.readline().strip()
    prev_CGI = ''
    while input_line:
        input_split = input_line.split('\t')
        input_chr = int(input_split[0].replace("chr","").replace("X","23").replace("Y","24"))
        input_pos = int(input_split[1])
        info = ['','','','','']
        
        input_distance = 999999999
        input_decision = ''
        for [info_CGI,info_start,info_end,info_size,info_class] in info_list[input_chr-1]:
            if input_pos >= info_start and input_pos <= info_end:
                info = ['o',info_CGI,'0',info_size,info_class]                
                break
            else:
                temp_distance1 = input_pos - info_start
                temp_distance2 = input_pos - info_end
                if min(abs(temp_distance1),abs(temp_distance2)) == abs(temp_distance1):
                    temp_distance = temp_distance1
                else:
                    temp_distance = temp_distance2
                
                if input_distance > abs(temp_distance):
                    input_distance = abs(temp_distance)
                    info = ['x',info_CGI,str(temp_distance),info_size,info_class]
        
        output_file.write(input_line+'\t' + '\t'.join(info) + '\n')
        input_line = input_file.readline().strip()
        
    
    
    input_file.close()
    output_file.close()

print 'THE END'



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/users/jhyouk/miniconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/users/jhyouk/miniconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/users/jhyouk/miniconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/users/jhyouk/miniconda2/lib/python2.7/inspect.py", line 1058, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/home/users/jhyouk/miniconda2/lib/python2.7/inspect.py", line 1018, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/home/users/jhyouk/miniconda2/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodul

IndexError: string index out of range

In [14]:
# Af mean (consider Af range later) exclude Y chromosome!
# Figure1f data

list_class = ['noCGI', 'NR_TSS', 'NR_intragenic', 'intergenic', 'NM_TSS', 'NM_intragenic', 'miscellaneous']
total_class = [0,0,0,0,0,0,0]

union_class = [[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

for i in range(1,24):
    print i
    input_file = file("revision_CpG_CtoT_PCAWG_CGI_reclassification_chr%s.txt" % str(i).replace("23","X").replace("24","Y"))
    input_line = input_file.readline().strip()
    input_line = input_file.readline().strip()

    while input_line:
        input_split = input_line.split('\t')
        
        if input_split[25] == 'x':
            input_class = 'noCGI'
        else:
            input_class = input_split[29]
            if len(input_class.split(',')) > 1:
                input_class = 'miscellaneous'

        c_index = 99
        
        if input_split[20].split(';')[1] == '.':
            input_pcawg = 0.0
        else:
            input_pcawg = float(input_split[20].split(';')[1]) 
            
        if input_split[21].split(';')[1] == '.':
            input_thousand = 0.0
        else:
            input_thousand = float(input_split[21].split(';')[1])             
        
        input_union = (input_pcawg + input_thousand) / 4327.0

        
        c_index = list_class.index(input_class)
        total_class[c_index] += 1
        
        if input_union == 0:
            union_class[c_index][0] += 1
        elif input_union <0.06:
            union_class[c_index][1] += 1
        elif input_union <0.12:
            union_class[c_index][2] += 1
        elif input_union <0.18:
            union_class[c_index][3] += 1           
        elif input_union <0.24:
            union_class[c_index][4] += 1
        else:
            union_class[c_index][5] += 1
            
        input_line = input_file.readline().strip()
    
print list_class
print total_class
print union_class

output_file = file("figure1f_af_group.txt","w")
output_file.write('\t'.join(list_class) + '\n')
output_file.write('\t'.join(map(str,total_class)) + '\n')
for j in range(0,6):
    for k in range(0,7):
        if k < 6:
            output_file.write(str(union_class[k][j])+ '\t')
        else:
            output_file.write(str(union_class[k][j])+ '\n')

output_file.close()


print 'THE END'


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
['noCGI', 'NR_TSS', 'NR_intragenic', 'intergenic', 'NM_TSS', 'NM_intragenic', 'miscellaneous']
[51703250, 116304, 73452, 597266, 1982770, 758164, 611056]
[[35295464, 15284146, 225953, 123312, 92011, 682364], [111252, 4704, 87, 37, 41, 183], [68772, 4350, 75, 45, 24, 186], [557438, 37104, 640, 378, 238, 1468], [1908076, 71176, 854, 451, 322, 1891], [667214, 87102, 912, 487, 345, 2104], [579803, 29906, 288, 155, 130, 774]]
THE END


In [7]:
# Af mean (consider Af range later) exclude Y chromosome!
# for confidence interval calculation

list_class = []
total_class = []

union_class = []

for i in range(1,24):
    print i
    input_file = file("revision_CpG_CtoT_PCAWG_CGI_reclassification_chr%s.txt" % str(i).replace("23","X").replace("24","Y"))
    input_line = input_file.readline().strip()
    input_line = input_file.readline().strip()

    while input_line:
        input_split = input_line.split('\t')
        
        if input_split[25] == 'x':
            input_class = 'noCGI'
        else:
            input_class = input_split[29]
            if len(input_class.split(',')) > 1:
                input_class = 'miscellaneous'

        c_index = 99
        
        if input_split[20].split(';')[1] == '.':
            input_pcawg = 0.0
        else:
            input_pcawg = float(input_split[20].split(';')[1]) 
            
        if input_split[21].split(';')[1] == '.':
            input_thousand = 0.0
        else:
            input_thousand = float(input_split[21].split(';')[1])             
        
        input_union = input_pcawg + input_thousand

        if input_class not in list_class:
            list_class.append(input_class)
            total_class.append(1)
            union_class.append([input_union])


        else:
            c_index = list_class.index(input_class)
            total_class[c_index] += 1
            union_class[c_index].append(input_union)
                
        input_line = input_file.readline().strip()
    
print list_class
print total_class
#print union_class


for i in range(0,len(union_class)):
    output_file = file("figure1e_af_variation_%s.txt" %list_class[i],"w")
    for j in union_class[i]:
        output_file.write(str(j)+'\n')
    output_file.close()
    
    


print 'THE END'



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
['noCGI', 'NR_TSS', 'NR_intragenic', 'intergenic', 'NM_TSS', 'NM_intragenic', 'miscellaneous']
[51703250, 116304, 73452, 597266, 1982770, 758164, 611056]
THE END


In [16]:
import numpy
a = [0,1,2,3]
print numpy.mean(a)

1.5
